In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cf
import netCDF4
import pandas as pd
import datetime
import pytz 
from tzwhere import tzwhere
import math
from suntime import Sun
import scipy as sci
from scipy import stats
from scipy.stats import norm

In [8]:
def get_data(len_data, NU_WU):  #len_data is the number of files i want to read in (e.g. 4 for 4h hourly data), NU_WU is a str and you can define if you want 'with urban' or 'no urban'
    ds_data = {}  
    for i in range(1, len_data + 1):
        filename = f'../det_data_all_2021/fc_DOM01_0{i:03d}_{NU_WU}_urban_grid.nc'
        try:
            ds_data[i] = xr.open_dataset(filename)
            print(f'Successfully opened {filename}')
        except FileNotFoundError:
            print(f'File not found: {filename}')
        except Exception as e:
            print(f'An error occurred while opening {filename}: {e}')
    
    return ds_data

def get_data_nights(len_data, NU_WU):  #len_data is the number of files i want to read in (e.g. 4 for 4h hourly data), NU_WU is a str and you can define if you want 'with urban' or 'no urban'
    ds_data = {}  
    for i in range(1, len_data + 1):
        filename = f'../det_data_nights_2021/fc_DOM01_0{i:03d}_{NU_WU}_urban_grid_nights.nc'
        try:
            ds_data[i] = xr.open_dataset(filename)
            print(f'Successfully opened {filename}')
        except FileNotFoundError:
            print(f'File not found: {filename}')
        except Exception as e:
            print(f'An error occurred while opening {filename}: {e}')
    
    return ds_data

def get_data_days(len_data, NU_WU):  #len_data is the number of files i want to read in (e.g. 4 for 4h hourly data), NU_WU is a str and you can define if you want 'with urban' or 'no urban'
    ds_data = {}  
    for i in range(1, len_data + 1):
        filename = f'../det_data_day_2021/fc_DOM01_0{i:03d}_{NU_WU}_urban_grid_days.nc'
        try:
            ds_data[i] = xr.open_dataset(filename)
            print(f'Successfully opened {filename}')
        except FileNotFoundError:
            print(f'File not found: {filename}')
        except Exception as e:
            print(f'An error occurred while opening {filename}: {e}')
    
    return ds_data

def get_data_ens(len_data, NU_WU, mem):
    ds_data = {}  
    for i in range(1, len_data + 1):
        filename = f'../ensemble_members_data_2021/{mem}/fc_DOM01_0{i:03d}_{mem}_{NU_WU}_urban_grid.nc'
        try:
            ds_data[i] = xr.open_dataset(filename)
            print(f'Successfully opened {filename}')
        except FileNotFoundError:
            print(f'File not found: {filename}')
        except Exception as e:
            print(f'An error occurred while opening {filename}: {e}')
    
    return ds_data

def prepare_sun(number_cities, data, file): #number of cities depends on input and % of landuse tiles, file defines in which hour you want to define since all hours are stored in one xarray dataset
    sun = {}
    for i in range(number_cities):
        sun[i] = Sun(data[file].clat[i].to_numpy()*180/math.pi, data[file].clon[i].to_numpy()*180/math.pi)
    
    return sun

def get_sunrise_sunset(number_cities, input_time, input_sun): #input_sun is the output of 'def prepare_sun'
    sunrise = {}
    sunset = {}
    for i in range(number_cities):
        sunrise[i] = input_sun[i].get_local_sunrise_time(input_time).time()
        sunrise[i] = pytz.utc.localize(sunrise[i])
    for i in range(number_cities):
        sunset[i] = input_sun[i].get_local_sunset_time(input_time).time()
        sunset[i] = pytz.utc.localize(sunset[i])
    
    return sunrise, sunset

def binary_daynight(number_cities, sunrise, sunset, hour):  
    time = datetime.time(hour)
    time = pytz.utc.localize(time)
    daynight = []
    
    for i in range(number_cities):
        if (sunset[i] < sunrise[i] and sunset[i]<= time <= sunrise[i]) or (sunset[i] > sunrise[i] and (time > sunset[i] or time < sunrise[i])):
            daynight.append('Night')
        else:
            daynight.append('Day')

    return daynight

def get_night(binary_data, data):
    data_night = {}

    for i, dataset in data.items():
        dataset = dataset.expand_dims(dim={"day_night": binary_data})
        
        data_night[i] = dataset.where(dataset['day_night'] != 'Day')
    
    return data_night, dataset


In [6]:
data_WU = get_data(120, 'URB')
data_NU = get_data(120, 'NU') 

Successfully opened ../det_data_all_2021/fc_DOM01_0001_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0002_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0003_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0004_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0005_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0006_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0007_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0008_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0009_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0010_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0011_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0012_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0013_URB_urban_grid.nc
Successfully opened ../det_data_all_2021/fc_DOM01_0

In [9]:
data_WU_nights = get_data_nights(120, 'URB')
data_NU_nights = get_data_nights(120, 'NU')

Successfully opened ../det_data_nights_2021/fc_DOM01_0001_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0002_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0003_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0004_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0005_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0006_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0007_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0008_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0009_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0010_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0011_URB_urban_grid_nights.nc
Successfully opened ../det_data_nights_2021/fc_DOM01_0012_URB_urban_grid_nights.nc
Succ

In [10]:
data_WU_days = get_data_days(120, 'URB')
data_NU_days = get_data_days(120, 'NU')

Successfully opened ../det_data_day_2021/fc_DOM01_0001_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0002_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0003_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0004_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0005_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0006_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0007_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0008_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0009_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0010_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0011_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0012_URB_urban_grid_days.nc
Successfully opened ../det_data_day_2021/fc_DOM01_0013_URB_urban

In [56]:
# List of 40 mem values (adjust as needed)
#mem_values = [f"mem{i:03d}" for i in range(1, 5)]

# Dictionary to store results for each mem value
#data_NU_mem = {}

# Loop over mem values
#for mem_value in mem_values:
#    directory_name = f"data_NU_{mem_value}"  # Create a unique directory name
#    data_NU_mem[directory_name] = get_data_ens(120, 'NU', mem_value)

In [39]:
#data_NU_mem001 = get_data_ens(120, 'NU', 'mem001')
#data_NU_mem002 = get_data_ens(120, 'NU', 'mem002')
#data_NU_mem003 = get_data_ens(120, 'NU', 'mem003')
#data_NU_mem004 = get_data_ens(120, 'NU', 'mem004')
#data_NU_mem005 = get_data_ens(120, 'NU', 'mem005')
#data_NU_mem006 = get_data_ens(120, 'NU', 'mem006')
#data_NU_mem007 = get_data_ens(120, 'NU', 'mem007')
#data_NU_mem008 = get_data_ens(120, 'NU', 'mem008')
#data_NU_mem009 = get_data_ens(120, 'NU', 'mem009')
#data_NU_mem010 = get_data_ens(120, 'NU', 'mem010')
#data_NU_mem011 = get_data_ens(120, 'NU', 'mem011')
#data_NU_mem012 = get_data_ens(120, 'NU', 'mem012')
#data_NU_mem013 = get_data_ens(120, 'NU', 'mem013')
#data_NU_mem014 = get_data_ens(120, 'NU', 'mem014')
#data_NU_mem015 = get_data_ens(120, 'NU', 'mem015')
#data_NU_mem016 = get_data_ens(120, 'NU', 'mem016')
#data_NU_mem017 = get_data_ens(120, 'NU', 'mem017')
#data_NU_mem018 = get_data_ens(120, 'NU', 'mem018')
#data_NU_mem019 = get_data_ens(120, 'NU', 'mem019')
#data_NU_mem020 = get_data_ens(120, 'NU', 'mem020')
#data_NU_mem021 = get_data_ens(120, 'NU', 'mem021')
#data_NU_mem022 = get_data_ens(120, 'NU', 'mem022')
#data_NU_mem023 = get_data_ens(120, 'NU', 'mem023')
#data_NU_mem024 = get_data_ens(120, 'NU', 'mem024')
#data_NU_mem025 = get_data_ens(120, 'NU', 'mem025')
#data_NU_mem026 = get_data_ens(120, 'NU', 'mem026')
#data_NU_mem027 = get_data_ens(120, 'NU', 'mem027')
#data_NU_mem028 = get_data_ens(120, 'NU', 'mem028')
#data_NU_mem029 = get_data_ens(120, 'NU', 'mem029')
#data_NU_mem030 = get_data_ens(120, 'NU', 'mem030')
#data_NU_mem031 = get_data_ens(120, 'NU', 'mem031')
#data_NU_mem032 = get_data_ens(120, 'NU', 'mem032')
#data_NU_mem033 = get_data_ens(120, 'NU', 'mem033')
#data_NU_mem034 = get_data_ens(120, 'NU', 'mem034')
#data_NU_mem035 = get_data_ens(120, 'NU', 'mem035')
#data_NU_mem036 = get_data_ens(120, 'NU', 'mem036')
#data_NU_mem037 = get_data_ens(120, 'NU', 'mem037')
#data_NU_mem038 = get_data_ens(120, 'NU', 'mem038')
#data_NU_mem039 = get_data_ens(120, 'NU', 'mem039')
#data_NU_mem040 = get_data_ens(120, 'NU', 'mem040')

Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0001_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0002_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0003_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0004_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0005_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0006_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0007_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0008_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0009_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem001/fc_DOM01_0010_mem001_NU_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/

In [216]:
#data_WU_mem001 = get_data_ens(120, 'URB', 'mem001')
#data_WU_mem002 = get_data_ens(120, 'URB', 'mem002')
#data_WU_mem003 = get_data_ens(120, 'URB', 'mem003')
#data_WU_mem004 = get_data_ens(120, 'URB', 'mem004')
#data_WU_mem005 = get_data_ens(120, 'URB', 'mem005')
#data_WU_mem006 = get_data_ens(120, 'URB', 'mem006')
#data_WU_mem007 = get_data_ens(120, 'URB', 'mem007')
#data_WU_mem008 = get_data_ens(120, 'URB', 'mem008')
#data_WU_mem009 = get_data_ens(120, 'URB', 'mem009')
#data_WU_mem010 = get_data_ens(120, 'URB', 'mem010')
#data_WU_mem011 = get_data_ens(120, 'URB', 'mem011')
#data_WU_mem012 = get_data_ens(120, 'URB', 'mem012')
#data_WU_mem013 = get_data_ens(120, 'URB', 'mem013')
#data_WU_mem014 = get_data_ens(120, 'URB', 'mem014')
#data_WU_mem015 = get_data_ens(120, 'URB', 'mem015')
#data_WU_mem016 = get_data_ens(120, 'URB', 'mem016')
#data_WU_mem017 = get_data_ens(120, 'URB', 'mem017')
#data_WU_mem018 = get_data_ens(120, 'URB', 'mem018')
#data_WU_mem019 = get_data_ens(120, 'URB', 'mem019')
data_WU_mem020 = get_data_ens(120, 'URB', 'mem020')
#data_WU_mem021 = get_data_ens(120, 'WU', 'mem021')
#data_WU_mem022 = get_data_ens(120, 'WU', 'mem022')
#data_WU_mem023 = get_data_ens(120, 'WU', 'mem023')
#data_WU_mem024 = get_data_ens(120, 'WU', 'mem024')
#data_WU_mem025 = get_data_ens(120, 'WU', 'mem025')
#data_WU_mem026 = get_data_ens(120, 'WU', 'mem026')
#data_WU_mem027 = get_data_ens(120, 'WU', 'mem027')
#data_WU_mem028 = get_data_ens(120, 'WU', 'mem028')
#data_WU_mem029 = get_data_ens(120, 'WU', 'mem029')
#data_WU_mem030 = get_data_ens(120, 'WU', 'mem030')
#data_WU_mem031 = get_data_ens(120, 'WU', 'mem031')
#data_WU_mem032 = get_data_ens(120, 'WU', 'mem032')
#data_WU_mem033 = get_data_ens(120, 'WU', 'mem033')
#data_WU_mem034 = get_data_ens(120, 'WU', 'mem034')
#data_WU_mem035 = get_data_ens(120, 'WU', 'mem035')
#data_WU_mem036 = get_data_ens(120, 'WU', 'mem036')
#data_WU_mem037 = get_data_ens(120, 'WU', 'mem037')
#data_WU_mem038 = get_data_ens(120, 'WU', 'mem038')
#data_WU_mem039 = get_data_ens(120, 'WU', 'mem039')
#data_WU_mem040 = get_data_ens(120, 'WU', 'mem040')

Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0001_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0002_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0003_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0004_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0005_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0006_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0007_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0008_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0009_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_data_2021/mem020/fc_DOM01_0010_mem020_URB_urban_grid.nc
Successfully opened ../ensemble_members_

In [41]:
ensem_WU = data_WU_mem001
ensem_NU = data_NU_mem001

In [44]:
#prepare sun #convert radiant to lat lon 

sun_WU_ensem = {}
sun_NU_ensem = {}

for i in range(1, 121):
    sun_WU_ensem[i] = prepare_sun(247, ensem_WU, i)
    
for i in range(1, 121):
    sun_NU_ensem[i] = prepare_sun(247, ensem_NU, i)

In [45]:
sunrise_WU = {}
sunrise_NU = {}

sunset_WU = {}
sunset_NU = {}

time_zone = datetime.date(2021,7,5)

for i in range(1, 121):
    sunrise_WU[i], sunset_WU[i] = get_sunrise_sunset(247, time_zone , sun_WU_ensem[i])

for i in range(1,121):
    sunrise_NU[i], sunset_NU[i] = get_sunrise_sunset(247, time_zone , sun_NU_ensem[i])

In [46]:
#create time_array for the hours because it is easier to calculate with. Added zero in the beginning because ICON output and Python have different indexing

time_array = []

for i in range(121):
    time_array.append(i % 24)

numbers_to_add = [0, 21, 22, 23]  # Numbers to add

# Use the insert method to add numbers to the beginning of the array
for num in reversed(numbers_to_add):
    time_array.insert(0, num)

In [47]:
#corrected version. Before the intervall between sunset and sunrise was wrongly defined for hours. Can delete the first version but just in case 

daynight_WU = {}
daynight_NU = {}

#time_array = [0,21, 22, 23, 24, 1, 2]

for i in range(1, 121):
    j = time_array[i]
    daynight_WU[i] = binary_daynight(247, sunrise_WU[i], sunset_WU[i], j)
    
for i in range(1, 121):
    j = time_array[i]
    daynight_NU[i] = binary_daynight(247, sunrise_NU[i], sunset_NU[i], j)

## from here on changes have to be made!

In [217]:
#Here the variable "daynight" is added to the original data_WU/NU dict/datasets

data_variable = {}

for i in range(1,121):
    
    data_variable[i] = xr.DataArray(daynight_WU[i], dims=('cell',), coords={'cell': data_WU_mem020[i]['cell']})
    data_WU_mem020[i]['daynight'] = data_variable[i]

# # for i in range(1,121):
    
#     data_variable[i] = xr.DataArray(daynight_NU[i], dims=('cell',), coords={'cell': data_NU_mem001[i]['cell']})
#     data_NU_mem001[i]['daynight'] = data_variable[i]

In [218]:
night_cells_WU = {}
#night_cells_NU = {}

for i in range(1,121):
    night_cells_WU[i] = data_WU_mem020[i].where(data_WU_mem020[i]['daynight'] == 'Night', drop=True)
    
# for i in range(1,121):
#     night_cells_NU[i] = data_NU_mem001[i].where(data_NU_mem001[i]['daynight'] == 'Night', drop=True)

In [219]:
day_cells_WU = {}
#day_cells_NU = {}

for i in range(1,121):
    if 'Day' in data_WU_mem020[i]['daynight'].values:
        day_cells_WU[i] = data_WU_mem020[i].where(data_WU_mem020[i]['daynight'] == 'Day', drop=True)
    
# for i in range(1,121):
#     if 'Day' in data_NU_mem001[i]['daynight'].values:
#         day_cells_NU[i] = data_NU_mem001[i].where(data_NU_mem001[i]['daynight'] == 'Day', drop=True)

In [220]:
# #and safe the new Day and Night Datsets so i don't have to do all of that at the beginning of each script 

# for i in range(1,121):
#     path = f'../ensemble_members_nights_2021/mem003/fc_DOM01_0{i:03d}_mem003_NU_urban_grid_nights.nc'
#     night_cells_NU[i].to_netcdf(path=path)

In [221]:
for i in range(1,121):
    path = f'../ensemble_members_nights_2021/mem020/fc_DOM01_0{i:03d}_mem020_URB_urban_grid_nights.nc'
    night_cells_WU[i].to_netcdf(path=path)

In [222]:
# for i in range(1,121):
#     path = f'../ensemble_members_days_2021/mem001/fc_DOM01_0{i:03d}_mem001_NU_urban_grid_days.nc'
#     day_cells_NU[i].to_netcdf(path=path)

In [223]:
for i in range(1,121):
    path = f'../ensemble_members_days_2021/mem020/fc_DOM01_0{i:03d}_mem020_URB_urban_grid_days.nc'
    day_cells_WU[i].to_netcdf(path=path)

In [224]:
print('hallo')

hallo
